In [1]:
from model import SVD

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math

from surprise import Dataset

In [ ]:
USERS_COUNT = 943
ITEMS_COUNT = 1682
THRESHOLD = 0.05

In [ ]:
def load_data():
    data = pd.DataFrame(Dataset.load_builtin("ml-100k").raw_ratings)
    data[0] = pd.to_numeric(data[0]) - 1
    data[1] = pd.to_numeric(data[1]) - 1
    del data[3]
    return data

In [ ]:
def load_similarities():
    similarities = pd.read_csv("artificial_ratings.csv")
    similarities['0'] = similarities['0'] - 1
    similarities['1'] = similarities['1'] - 1

    similarities_arr = np.zeros((ITEMS_COUNT, ITEMS_COUNT))
    for _, row in similarities.iterrows():
        if int(row['0']) != int(row['1']):
            similarities_arr[int(row['0']), int(row['1'])] = row['2']
        
    return similarities_arr

In [ ]:
similarities = load_similarities()

In [ ]:
artificial = np.zeros((USERS_COUNT, ITEMS_COUNT))
for item in range(ITEMS_COUNT):
    for user in range(USERS_COUNT):
        rating = 0
        all_user_ratings = train_set[train_set[:, 0] == user]
        sum_sim = 0
        for u, i, r in all_user_ratings:
            sim1 = similarities[item, int(i)]
            if (sim1 < 0.05):
                continue
            rating = rating + r * sim1
            sum_sim = sum_sim + sim1
        if sum_sim == 0:
            artificial[user, item] = -1
        else:
            artificial[user, item] = rating / sum_sim

dataframe = pd.DataFrame(columns=[0, 1, 2])
artificial_data = []
for item in range(ITEMS_COUNT):
    for user in range(USERS_COUNT):
        if (artificial[user, item] != -1):
            artificial_data.append([user, item, artificial[user, item]])

In [ ]:
data = load_data()

In [ ]:
actual_data = data.values

In [ ]:
train_set, _ = train_test_split(actual_data, test_size=0.95)
sparcity = 1 - (len(train_set) / (USERS_COUNT * ITEMS_COUNT))
print("Sparcitiy: %.4f" % sparcity)
train_set, test_set = train_test_split(train_set, test_size=.20)

In [ ]:
artificial_data = np.array(artificial_data)

In [ ]:
artificial_data_l = np.column_stack((artificial_data, np.zeros(artificial_data.shape[0])))
train_set_l = np.column_stack((artificial_data, np.ones(artificial_data.shape[0])))

In [ ]:
labled_train_set = np.vstack((artificial_data_l, train_set_l))
np.random.shuffle(labled_train_set)

In [ ]:
algo = SVD(verbose=True)
algo.fit(labled_train_set)

In [ ]:
algo = SVD(verbose=True)
algo.fit(np.array(artificial_data))

In [ ]:
algo = SVD(verbose=True)
algo.fit(train_set)

In [ ]:
predictions = [(r_ui_trans, algo.estimate(uid, iid)) for (uid, iid, r_ui_trans) in test_set]

In [ ]:
predictions = np.array(predictions)

In [ ]:
math.sqrt(mean_squared_error(predictions[:, 0], predictions[:, 1]))

In [ ]:
predictions = [(r_ui_trans, algo.estimate(uid, iid)) for (uid, iid, r_ui_trans) in actual_data]